In [2]:
from num2words import num2words

def numero_a_texto(numero):
    # Separar la parte entera de la parte decimal
    parte_entera, parte_decimal = str(numero).split(',')
    
    # Convertir la parte entera a texto
    texto_parte_entera = num2words(int(parte_entera.replace('.', '')), lang='es')
    
    # Convertir la parte decimal a texto
    texto_parte_decimal = f"{int(parte_decimal):02d}/100"
    
    # Formatear el resultado
    resultado = f"{texto_parte_entera} {texto_parte_decimal}"
    
    # Capitalizar la primera letra
    resultado = resultado.capitalize()
    
    return resultado

# Ejemplo de uso
numero = "252352,56"
texto = numero_a_texto(numero)
print(texto)


Un millón ciento veintitrés mil ciento veinticinco 52/100
